<a href="https://colab.research.google.com/github/JacksonXi/optimisation/blob/main/flexible_control.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import shutil
import sys
import os.path
import numpy as np
import pandas as pd

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("cbc") or os.path.isfile("cbc")):
    if "google.colab" in sys.modules:
        !apt-get install -y -qq coinor-cbc
    else:
        try:
            !conda install -c conda-forge coincbc 
        except:
            pass

assert(shutil.which("cbc") or os.path.isfile("cbc"))
    
from pyomo.environ import *


     |████████████████████████████████| 8.9 MB 7.3 MB/s 
     |████████████████████████████████| 49 kB 5.0 MB/s 
Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 160837 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unp

In [2]:
!git clone https://github.com/JacksonXi/optimisation

Cloning into 'optimisation'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 49 (delta 8), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (49/49), done.


In [3]:
cd /content/optimisation/flexible/

/content/optimisation/flexible


**non-controlable energy < generated energy < (non-controlable energy + flexible energy)**

In [4]:
Flexibleindex = pd.read_csv("flexibleIndex.csv")
Consumer = list(Flexibleindex['Unnamed: 0'])
Consumer1 = list(Flexibleindex['Unnamed: 0'])
Consumer

['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10']

In [5]:
flexibleindex = pd.read_csv("flexibleIndex.csv", index_col=0)
flexibleindex

,TV,Heater,WashingMachine,Dryer,Fridger,Airconditioner,InductionCooker
C1,0,0,1,1,0,1,0
C2,1,0,1,0,0,1,0
C3,1,0,0,1,0,1,0
C4,0,1,0,1,1,0,1
C5,1,0,1,0,1,0,0
C6,0,1,0,1,0,1,0
C7,0,0,0,1,0,1,1
C8,1,0,0,1,0,0,1
C9,0,0,0,1,0,0,0
C10,1,1,1,0,0,0,1


In [6]:
power = pd.read_csv('power.csv', index_col=0)
power

,TV,Heater,WashingMachine,Dryer,Fridger,Airconditioner,InductionCooker
C1,2,4,4,8,2,5,5
C2,2,7,4,5,1,3,5
C3,2,5,2,5,2,4,4
C4,2,8,3,5,1,4,4
C5,2,5,4,5,2,3,6
C6,1,6,2,6,1,5,5
C7,1,7,4,5,1,5,6
C8,1,7,2,6,1,3,4
C9,1,4,4,6,1,5,5
C10,2,8,2,5,1,4,5


In [7]:
appliances = list(flexibleindex.keys())
appliances

['TV',
 'Heater',
 'WashingMachine',
 'Dryer',
 'Fridger',
 'Airconditioner',
 'InductionCooker']

In [8]:
bid = pd.read_csv('bid.csv', index_col=0)
bid

,bid
C1,18
C2,15
C3,19
C4,21
C5,22
C6,20
C7,17
C8,16
C9,18
C10,21


In [9]:
n = len(bid)
consumer = [] 
while n != 0:
    max = Consumer[0]
    for i in range(0,n):
      if bid['bid'][max] < bid['bid'][Consumer[i]]:
          max = Consumer[i]
      else:
          max = max
    consumer.append(max)      
    Consumer.remove(max)
    n -= 1
print(Consumer)
print(consumer)

[]
['C5', 'C4', 'C10', 'C6', 'C3', 'C1', 'C9', 'C7', 'C8', 'C2']


In [10]:
generatedEnergy = pd.read_csv('generated energy.csv')
Prosumer = list(generatedEnergy['Unnamed: 0'])
Prosumer1 = list(generatedEnergy['Unnamed: 0'])
Prosumer

['P1', 'P2', 'P3', 'P4', 'P5', 'P6']

In [11]:
generated_energy = pd.read_csv('generated energy.csv', index_col=0)
generated_energy

,generated energy
P1,31
P2,25
P3,4
P4,23
P5,29
P6,32


In [12]:
n = len(generated_energy)
prosumer = [] 
while n != 0:
    max = Prosumer[0]
    for i in range(0,n):
      if generated_energy['generated energy'][max] < generated_energy['generated energy'][Prosumer[i]]:
          max = Prosumer[i]
      else:
          max = max
    prosumer.append(max)      
    Prosumer.remove(max)
    n -= 1
print(prosumer)
print(Prosumer)

['P6', 'P1', 'P5', 'P2', 'P4', 'P3']
[]


**Calaulate the total generated energy**

In [13]:
TotalEnergy = 0

for p in prosumer:
    TotalEnergy += generated_energy['generated energy'][p]

TotalEnergy

144

**Calculate the total energy of non-controlable loads and revenue** 





In [14]:
demand = pd.read_csv('demand.csv', index_col=0)
print(demand)

d = 0
for c in consumer:
    d += demand['demand'][c]*bid['bid'][c]
print(d)

D = 0
for c in consumer:
    D += demand['demand'][c]
print(D)

     demand
C1        3
C2       13
C3        1
C4        1
C5        5
C6        3
C7        4
C8        6
C9        8
C10       5
872
49


In [15]:
model = ConcreteModel()
model.dual = Suffix(direction=Suffix.IMPORT)

model.switches = Var(appliances, consumer, domain = Binary)

model.revenue = Objective(expr = d + sum(flexibleindex[a][c]*power[a][c]*bid['bid'][c]*model.switches[a, c] for c in consumer for a in appliances),
                          sense = maximize)

model.supply = Constraint(expr = (D + sum(flexibleindex[a][c]*power[a][c]*model.switches[a, c] for c in consumer for a in appliances)) <= TotalEnergy)


results = SolverFactory('cbc').solve(model)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 2762.0
  Upper bound: 2762.0
  Number of objectives: 1
  Number of constraints: 1
  Number of variables: 30
  Number of binary variables: 30
  Number of integer variables: 30
  Number of nonzeros: 30
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.01
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and boun

In [16]:
for c in Consumer1:
    for a in appliances:
        if model.switches[a,c]() == 1:
          print("{:<5s} {:<2s} {:<12} {:<15} {:<8}".format('User', c , ': The state of', a,'is ON'))
        elif model.switches[a,c]() == 0:
          print("{:<5s} {:<2s} {:<12} {:<15} {:<8}".format('User', c , ': The state of', a,'is OFF'))
        else:
          print("{:<5s} {:<2s} {:<12} {:<15} {:<8}".format('User', c , ': The state of', a,'is UNCONTROLABLE'))
    print('\n')

User  C1 : The state of TV              is UNCONTROLABLE
User  C1 : The state of Heater          is UNCONTROLABLE
User  C1 : The state of WashingMachine  is ON   
User  C1 : The state of Dryer           is ON   
User  C1 : The state of Fridger         is UNCONTROLABLE
User  C1 : The state of Airconditioner  is ON   
User  C1 : The state of InductionCooker is UNCONTROLABLE


User  C2 : The state of TV              is OFF  
User  C2 : The state of Heater          is UNCONTROLABLE
User  C2 : The state of WashingMachine  is OFF  
User  C2 : The state of Dryer           is UNCONTROLABLE
User  C2 : The state of Fridger         is UNCONTROLABLE
User  C2 : The state of Airconditioner  is OFF  
User  C2 : The state of InductionCooker is UNCONTROLABLE


User  C3 : The state of TV              is ON   
User  C3 : The state of Heater          is UNCONTROLABLE
User  C3 : The state of WashingMachine  is UNCONTROLABLE
User  C3 : The state of Dryer           is ON   
User  C3 : The state of Fridger   

**Calculating the optimized energy demand**

In [17]:
for c in consumer:
    for a in appliances:
        if model.switches[a, c]() != None:
            demand['demand'][c] += power[a][c]*model.switches[a, c]()

demand

,demand
C1,20
C2,13
C3,12
C4,19
C5,13
C6,20
C7,4
C8,7
C9,14
C10,22


In [18]:
Model = ConcreteModel()
Model.dual = Suffix(direction=Suffix.IMPORT)

Model.transport = Var(prosumer, consumer , domain = NonNegativeReals)

Model.revenue = Objective(expr = sum(Model.transport[p, c]*bid['bid'][c] for p in prosumer for c in consumer), 
                           sense = maximize)

Model.supply = ConstraintList()
for p in prosumer:
      Model.supply.add(sum(Model.transport[p, c] for c in consumer) <= generated_energy['generated energy'][p])

Model.demand = ConstraintList()
for c in consumer:
      Model.demand.add(sum(Model.transport[p, c] for p in prosumer) == demand['demand'][c])

results = SolverFactory('cbc').solve(Model)
results.write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 2762.0
  Upper bound: 2762.0
  Number of objectives: 1
  Number of constraints: 17
  Number of variables: 61
  Number of nonzeros: 60
  Sense: maximize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of create

In [19]:
for p in prosumer:
    for c in consumer:
        print(p, c, Model.transport[p,c]())

P6 C5 13.0
P6 C4 0.0
P6 C10 0.0
P6 C6 0.0
P6 C3 0.0
P6 C1 12.0
P6 C9 0.0
P6 C7 0.0
P6 C8 7.0
P6 C2 0.0
P1 C5 0.0
P1 C4 10.0
P1 C10 0.0
P1 C6 0.0
P1 C3 7.0
P1 C1 0.0
P1 C9 14.0
P1 C7 0.0
P1 C8 0.0
P1 C2 0.0
P5 C5 0.0
P5 C4 0.0
P5 C10 0.0
P5 C6 20.0
P5 C3 5.0
P5 C1 0.0
P5 C9 0.0
P5 C7 4.0
P5 C8 0.0
P5 C2 0.0
P2 C5 0.0
P2 C4 9.0
P2 C10 3.0
P2 C6 0.0
P2 C3 0.0
P2 C1 0.0
P2 C9 0.0
P2 C7 0.0
P2 C8 0.0
P2 C2 13.0
P4 C5 0.0
P4 C4 0.0
P4 C10 19.0
P4 C6 0.0
P4 C3 0.0
P4 C1 4.0
P4 C9 0.0
P4 C7 0.0
P4 C8 0.0
P4 C2 0.0
P3 C5 0.0
P3 C4 0.0
P3 C10 0.0
P3 C6 0.0
P3 C3 0.0
P3 C1 4.0
P3 C9 0.0
P3 C7 0.0
P3 C8 0.0
P3 C2 0.0


In [20]:
if 'ok' == str(results.Solver.status):
    print("Total Community Revenue = ",model.revenue())
    print("\nTransport Table:")
    for p in Prosumer1:
        for c in Consumer1:
            if Model.transport[p,c]() > 0:
                print("Transport from ", p," to ", c, ":", Model.transport[p,c](),'KWh')
else:
    print("No Valid Solution Found")

Total Community Revenue =  2762.0

Transport Table:
Transport from  P1  to  C3 : 7.0 KWh
Transport from  P1  to  C4 : 10.0 KWh
Transport from  P1  to  C9 : 14.0 KWh
Transport from  P2  to  C2 : 13.0 KWh
Transport from  P2  to  C4 : 9.0 KWh
Transport from  P2  to  C10 : 3.0 KWh
Transport from  P3  to  C1 : 4.0 KWh
Transport from  P4  to  C1 : 4.0 KWh
Transport from  P4  to  C10 : 19.0 KWh
Transport from  P5  to  C3 : 5.0 KWh
Transport from  P5  to  C6 : 20.0 KWh
Transport from  P5  to  C7 : 4.0 KWh
Transport from  P6  to  C1 : 12.0 KWh
Transport from  P6  to  C5 : 13.0 KWh
Transport from  P6  to  C8 : 7.0 KWh


In [21]:
n = 0
if 'ok' == str(results.Solver.status):
    print("Personal Revenue:\n")
    for p in prosumer:
        for c in consumer:
            if Model.transport[p,c]() > 0:
                print("Balance from ", c," to ", p, ":", Model.transport[p,c]() * bid['bid'][c])
else:
    print("No Valid Solution Found")

Personal Revenue:

Balance from  C5  to  P6 : 286.0
Balance from  C1  to  P6 : 216.0
Balance from  C8  to  P6 : 112.0
Balance from  C4  to  P1 : 210.0
Balance from  C3  to  P1 : 133.0
Balance from  C9  to  P1 : 252.0
Balance from  C6  to  P5 : 400.0
Balance from  C3  to  P5 : 95.0
Balance from  C7  to  P5 : 68.0
Balance from  C4  to  P2 : 189.0
Balance from  C10  to  P2 : 63.0
Balance from  C2  to  P2 : 195.0
Balance from  C10  to  P4 : 399.0
Balance from  C1  to  P4 : 72.0
Balance from  C1  to  P3 : 72.0


In [22]:
n = 0
if 'ok' == str(results.Solver.status):
    print("Personal Revenue:\n")
    for p in prosumer:
      for c in consumer:
          n += Model.transport[p, c]() * bid['bid'][c] 
      print("The revenue of",p ,":" , n)
      n = 0
else:
    print("No Valid Solution Found")

Personal Revenue:

The revenue of P6 : 614.0
The revenue of P1 : 595.0
The revenue of P5 : 563.0
The revenue of P2 : 447.0
The revenue of P4 : 471.0
The revenue of P3 : 72.0


In [23]:
n = 0
if 'ok' == str(results.Solver.status):
    print("The exported electricity to the grid:\n")
    for p in prosumer:
        n = generated_energy['generated energy'][p]
        for c in consumer:
            n -= Model.transport[p, c]()
        if n > 0 :
            print(p, "will export ",n , "KWh to the grid") 

The exported electricity to the grid:



In [24]:
n = 0
if 'ok' == str(results.Solver.status):
    print("Personal Revenue:\n")
    for c in Consumer1:
      for p in Prosumer1:
          n += Model.transport[p, c]()
      print("The received energy of",c ,":" , n)
      n = 0
else:
    print("No Valid Solution Found")

Personal Revenue:

The received energy of C1 : 20.0
The received energy of C2 : 13.0
The received energy of C3 : 12.0
The received energy of C4 : 19.0
The received energy of C5 : 13.0
The received energy of C6 : 20.0
The received energy of C7 : 4.0
The received energy of C8 : 7.0
The received energy of C9 : 14.0
The received energy of C10 : 22.0
